In [2]:
import pandas as pd
import datetime
import re
import xarray as xr
import numpy as np 
import xroms
import xarray as xr
import pyresample 
import metpy
import matplotlib.pyplot as plt
import os, glob

In [3]:
# Function transfroming the txt file to a pandas dataframe:
def txt_to_pd(txtfile,LAT,LON):
    
    # Set whcih columns to keep and set new names:
    columns_to_keep = ['T_degC', 'T_qf', 'S', 'S_qf', 'Date', 'Time'] 
    new_column_names = ['TEMP','TEMP_QC','PSAL','PSAL_QC','Date','Time']
    
    # Change the names of the months to English from Norwegian
    parse = lambda x: datetime.datetime.strptime(x.replace('Des', 'Dec').replace('Mai', 'May').replace('Okt', 'Oct'), '%d %b %Y %H:%M:%S')

    #df = pd.read_csv(txtfile, delimiter='\t', usecols=columns_to_keep, parse_dates={"Datetime" : ['Date', 'Time']}, date_parser = parse)
    df = pd.read_csv(txtfile, delimiter='\t', usecols=columns_to_keep)
    df.columns = new_column_names
    
    # Combine Date and Time columns and strip any leading or trailing whitespace
    df['TIME'] = (df['Date'] + ' ' + df['Time']).str.strip()
    
    # Apply the custom parsing function
    df['TIME'] = df['TIME'].apply(parse)
    df.drop(columns=['Date', 'Time'], inplace=True)
    
    # Add depth:
    df['DEPTH'] = get_depth(txtfile)
    df['LAT'] = LAT
    df['LON'] = LON
    return(df)


def netcdf_to_pd(txt):
    # Load the NetCDF file using xarray
    ds = xr.open_dataset(txt)

    # Select the variables you are interested in
    variables = ['TEMP', 'TEMP_QC', 'PSAL', 'PSAL_QC']

    # Initialize an empty DataFrame to merge into
    df_combined = pd.DataFrame()

    # Loop over each variable to process and merge
    for var in variables:

        # Select the variable data
        data = ds[var]
    
        # Stack the depth and time dimensions into a MultiIndex
        stacked_data = data.stack(points=('DEPTH', 'TIME'))
    
    
        # Convert the stacked DataArray to a pandas DataFrame
        df = stacked_data.to_dataframe()
            
        # Temporarily rename columns to avoid conflicts
        df.rename(columns={'DEPTH': 'Depth_col', 'TIME': 'Time_col', 'LONGITUDE': 'LON_col'}, inplace=True)
        #        
        df = df.reset_index()
        df.drop(columns=['Depth_col', 'Time_col'], inplace=True)
        #    
        # If df_combined is empty, initialize it with the current DataFrame
        if df_combined.empty:
            df_combined = df
        else:
            # Merge the current DataFrame with the combined DataFrame
            df_combined = pd.merge(df_combined, df, on=['DEPTH', 'TIME'], how='outer')  
    df_combined['LON'] = ds['LONGITUDE'].values[0]
    df_combined['LAT'] = ds['LATITUDE'].values[0]
    return(df_combined)        

In [4]:
def get_depth(filename):
    
    # Use a regular expression to extract the depth value
    depth_match = re.search(r'_(\d+)m', filename)
    if depth_match:
        depth_value = int(depth_match.group(1))
    else:
        depth_value = None
    return(depth_value)   

def get_Zindices(ds,X,Y,target_depth): 
    
    # ds er en xromsfil, gridfil fra norshelf
    ds = ds.isel(xi_rho = X, eta_rho = Y)

    # Extract the z_rho values at the given point
    z_rho_values = ds.z_rho.values
    s_rho_values = ds.s_rho.values

    diff = np.abs(z_rho_values - target_depth)
    
    # Find the indices of the two smallest differences
    closest_indices = diff.argsort()[0][:2]
    
    return(closest_indices)

def find_time_index(r_time,ocean_time):
    # r_time er datoen vi vil ha
    days_since_1970 = (r_time - datetime.datetime(1970,1,1,0,0,0)).total_seconds()
    ocean_time = pd.to_datetime(ocean_time)
    # Convert ocean_time
    ocean_time_since_1970 = (ocean_time - datetime.datetime(1970,1,1,0,0,0)).total_seconds()
    index = np.abs(ocean_time_since_1970 - days_since_1970).argmin()
    return(index)

def get_XYpositions(filename, lons, lats):
    
    fh = xr.open_dataset(filename)
    x   = np.linspace(0, fh.lat_rho.values.shape[1]-1, fh.lat_rho.values.shape[1])
    y   = np.linspace(0, fh.lat_rho.values.shape[0]-1, fh.lat_rho.values.shape[0])
    xi  = np.zeros_like(fh.lon_rho.values)
    yi  = np.zeros([fh.lon_rho.values.shape[1], fh.lon_rho.values.shape[0]])
    xi[:,:] = x
    yi[:,:] = y
    yi  = np.swapaxes(yi, 1, 0)

    # First I define the wet points of the field as the lon,lat values with mask_rho==1 
    sea_def = pyresample.geometry.SwathDefinition(lons= fh.lon_rho.values[np.where(fh.mask_rho)], lats = fh.lat_rho.values[np.where(fh.mask_rho)])

    # Second, the full grid definiton (our target domain):
    orig_def = pyresample.geometry.SwathDefinition(lons=lons, lats=lats)

    # Then I fill the temperature field by the nearest neighbour approace.
    # Note that only wet points are used as input. 

    # The radius of influence sets a limit (in meters) for how far away a true value can be from the point that will be filled

    ypos = pyresample.kd_tree.resample_nearest(sea_def, yi[np.where(fh.mask_rho)], \
                               orig_def, radius_of_influence=2400)

    xpos = pyresample.kd_tree.resample_nearest(sea_def, xi[np.where(fh.mask_rho)], \
                               orig_def, radius_of_influence=2400)
    return np.array([int(x) for x in xpos]), np.array([int(y) for y in ypos])

In [5]:

df_d1 = "/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment1/StationM_2021_hydrography.nc"

# Lese inn tekstfilene for deployment 2 (d2) ved 500, 800, 1000, 1200 m:

txtfile_d2_500m  = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment2/StaM_SBE_20211127_500m.txt'
txtfile_d2_800m  = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment2/StaM_SBE_20211127_800m.txt'
txtfile_d2_1000m = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment2/StaM_SBE_20211127_1000m.txt'
txtfile_d2_1200m = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment2/StaM_SBE_20211127_1200m.txt'
txtfile_d2_2000m = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment2/StaM_SBE_20211127_2000m.txt'

# Lese inn tekstfilene for deployment 3 (d3) ved 500, 800, 1000, 1200 m:

txtfile_d3_500m  = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment3/StaM_SBE_20221125_500m.txt'
txtfile_d3_800m  = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment3/StaM_SBE_20221125_800m.txt'
txtfile_d3_1000m = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment3/StaM_SBE_20221125_1000m.txt'
txtfile_d3_1200m = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment3/StaM_SBE_20221125_1200m.txt'
txtfile_d3_2000m = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment3/StaM_SBE_20221125_2000m.txt'

# Lese inn filer for Mooring South Cape:

south_cape_df = "/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/SouthCape/T_S_SouthCape.csv"


txtfile = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment2/StaM_SBE_20211127_1000m.txt'

ds = txt_to_pd(txtfile_d2_500m,66.015,1.983)

deployment1 = '/lustre/storeB/project/fou/hi/projects/NorEmso/Observations/moorings/StationM/Deployment1/StationM_2021_hydrography.nc'
ds2 = netcdf_to_pd(deployment1)

gridfile = '/lustre/storeB/project/fou/hi/oper/norshelf/static_inputfiles/norshelf_2.4_vert_grd.nc'
x,y = get_XYpositions(gridfile, ds2.LON.values, ds2.LAT.values)
ds2['X'] = x
ds2['Y'] = y

x,y = get_XYpositions(gridfile, ds.LON.values, ds.LAT.values)
ds['X'] = x
ds['Y'] = y


ds2['TIME'] = pd.to_datetime(ds2['TIME'])
# Round the TIME column to the nearest hour
ds2['TIME'] = ds2['TIME'].dt.round('H')

ds2['DAY'] = ds2['TIME'].dt.date

ds['TIME'] = pd.to_datetime(ds['TIME'])
# Round the TIME column to the nearest hour
ds['TIME'] = ds['TIME'].dt.round('H')

ds['DAY'] = ds['TIME'].dt.date
"""
# Test-data - sjekk om det funker 
ds2=ds2.head(50)
# Test-data
ds=ds.head(50)
print(ds.head(5))
print(" ds2 ")
print(ds2.head(5))#"""

'\n# Test-data - sjekk om det funker \nds2=ds2.head(50)\n# Test-data\nds=ds.head(50)\nprint(ds.head(5))\nprint(" ds2 ")\nprint(ds2.head(5))#'

In [6]:


# Define a function to extract data for each group
def extract_data_for_group(group,dsG,var='temp'):
    date = group['DAY'].iloc[0]  # All rows in the group have the same date
    year = date.strftime('%Y')
    month = date.strftime('%m')
    day = date.strftime('%d')
    
    file_path = f'https://thredds.met.no/thredds/dodsC/sea_norshelf_files/{year}/{month}/norshelf_qck_an_{year}{month}{day}T00Z.nc'
    # Read the file and extract data (assuming file has some structured data)
    
    #extracted_data = []
    
    try:
        with xr.open_dataset(file_path) as ds:
            # Selcect variables: 
            ds = ds.get([var])
            # Here we simulate extracting relevant data from the file for each x, y
            extracted_data = []
            for _, row in group.iterrows():
                x, y = row['X'], row['Y']
                
                #fine time-index
                r_time = row['TIME']
                ocean_time = ds.ocean_time.values
                index = find_time_index(r_time,ocean_time)
                
                #find depth index to interpolate over:
                depth = row['DEPTH']
                indices = get_Zindices(dsG,x,y,depth*-1)
                
                # Data extraction based on x, y, time and s_rho
                # OBS: This is just and example of extraction, have to find the correct ocean_time and s_rho!!!!
                temp1 = ds.isel(ocean_time=index,s_rho = indices[0], xi_rho = x, eta_rho = y)[var].values
                temp2 = ds.isel(ocean_time=index,s_rho = indices[1], xi_rho = x, eta_rho = y)[var].values
                m = np.array([temp1,temp2])
                extracted_data.append(np.mean(m))
            #return extracted_data
    except:
        extracted_data = []
        for _, row in group.iterrows():
            extracted_data.append(np.nan)
            
    return extracted_data   
        

        
gridfile = '/lustre/storeB/project/fou/hi/oper/norshelf/static_inputfiles/norshelf_2.4_vert_grd.nc' #denne er fast
# Load your ROMS dataset
dsG = xr.open_dataset(gridfile)
# Initialize the ROMS dataset and create the grid object
dsG, xgrid = xroms.roms_dataset(dsG, include_cell_volume=True, include_Z0=True)
# Associate the dataset with the grid
dsG.xroms.set_grid(xgrid)




In [7]:
# Group by the TIME column and apply the function to each group
grouped = ds2.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'temp'))
print("Ferdiiiii!!")

Ferdiiiii!!


In [8]:
# Flatten the grouped data into the original DataFrame
ds2['TEMP_MOD'] = [item for sublist in grouped for item in sublist]
print("ferdiiii! :P")

ferdiiii! :P


In [9]:
# Group by the TIME column and apply the function to each group
grouped = ds2.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'salt'))
print("ferdiiii! :P")

ferdiiii! :P


In [10]:
# Flatten the grouped data into the original DataFrame
ds2['SALT_MOD'] = [item for sublist in grouped for item in sublist]
print("ferdiiii! :P")

ferdiiii! :P


In [11]:
"""

# Group by the TIME column and apply the function to each group
grouped = ds2.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'temp'))

    # Flatten the grouped data into the original DataFrame
ds2['TEMP_MOD'] = [item for sublist in grouped for item in sublist]

# Group by the TIME column and apply the function to each group
grouped = ds2.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'salt'))

    # Flatten the grouped data into the original DataFrame
ds2['SALT_MOD'] = [item for sublist in grouped for item in sublist]
"""

"\n\n# Group by the TIME column and apply the function to each group\ngrouped = ds2.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'temp'))\n\n    # Flatten the grouped data into the original DataFrame\nds2['TEMP_MOD'] = [item for sublist in grouped for item in sublist]\n\n# Group by the TIME column and apply the function to each group\ngrouped = ds2.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'salt'))\n\n    # Flatten the grouped data into the original DataFrame\nds2['SALT_MOD'] = [item for sublist in grouped for item in sublist]\n"

In [12]:
# hen do the same for ds

In [13]:
# Group by the TIME column and apply the function to each group
grouped = ds.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'temp'))

    # Flatten the grouped data into the original DataFrame
ds['TEMP_MOD'] = [item for sublist in grouped for item in sublist]

# Group by the TIME column and apply the function to each group
grouped = ds.groupby('DAY').apply(lambda group: extract_data_for_group(group,dsG,'salt'))

    # Flatten the grouped data into the original DataFrame
ds['SALT_MOD'] = [item for sublist in grouped for item in sublist]


In [14]:
#Save csv file
ds.to_csv('StationM_500m_Dep2_norshelf.txt', float_format='%.3f', index=False)


In [15]:
#Save csv file
ds2.to_csv('StationM_Dep1_norshelf.txt', float_format='%.3f', index=False)

,TEMP,TEMP_QC,PSAL,PSAL_QC,TIME,DEPTH,LAT,LON,X,Y,DAY,temp,salt,TEMP_MOD,SALT_MOD
0,-0.4358,1,34.911854,1,2021-11-27 12:00:00,1000,66.015,1.983,406,340,2021-11-27,0.800000,34.9175,0.800000,34.9175
1,-0.4623,1,34.912240,1,2021-11-27 13:00:00,1000,66.015,1.983,406,340,2021-11-27,0.795500,34.9175,0.795500,34.9175
2,-0.4573,1,34.910988,1,2021-11-27 14:00:00,1000,66.015,1.983,406,340,2021-11-27,0.790500,34.9170,0.790500,34.9170
3,-0.4505,1,34.910341,1,2021-11-27 15:00:00,1000,66.015,1.983,406,340,2021-11-27,0.788000,34.9170,0.788000,34.9170
4,-0.4569,1,34.910742,1,2021-11-27 16:00:00,1000,66.015,1.983,406,340,2021-11-27,0.788000,34.9170,0.788000,34.9170
5,-0.4477,1,34.910972,1,2021-11-27 17:00:00,1000,66.015,1.983,406,340,2021-11-27,0.788000,34.9170,0.788000,34.9170
6,-0.4445,1,34.912828,1,2021-11-27 18:00:00,1000,66.015,1.983,406,340,2021-11-27,0.787000,34.9170,0.787000,34.9170
7,-0.4573,1,34.910747,1,2021-11-27 19:00:00,1000,66.015,1.983,406,340,2021-11-27,0.784500,34.9170,0.784500,34.9170
8,-0.4678,1,34.910478,1,2021-11-27 20:00:00,1000,66.015,1.983,406,340,2021-11-27,0.779500,34.9170,0.779500,34.9170
9,-0.4424,1,34.909580,1,2021-11-27 21:00:00,1000,66.015,1.983,406,340,2021-11-27,0.775499,34.9170,0.775499,34.9170
